In [1]:
import time
import numpy as np
import h5py
import matplotlib.pyplot as plt
import scipy
from scipy import ndimage
from dnn_utils import *
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
import os

path, dirs, cat_files_train = next(os.walk("training_set/cats/"))
file_count_cat = len(cat_files_train)
print(file_count_cat)

path, dirs, dog_files_train = next(os.walk("training_set/dogs/"))
file_count_dog = len(dog_files_train)
print(file_count_dog)
    

4001
4001


In [67]:
n_cat = 100
n_dog = 100

#Variables
m = n_cat + n_dog               # Number of train ex.
print(m)
num_px = 64
label_y = np.zeros((1,m))
X_train = np.zeros((12288,m))
# Input Dataset from the folder

for i in range(m):
#     print(i%n_cat)
    if i<n_cat:
        my_image = "cat."+str(i+1)+".jpg" # change this to the name of your image file 
        fname = "training_set/cats/" + my_image
        image = np.array(ndimage.imread(fname, flatten=False))
        my_image = scipy.misc.imresize(image, size=(num_px,num_px)).reshape((num_px*num_px*3,1))
        label_y[0][i] = 1 # the true class of your image (1 -> cat, 0 -> non-cat)
        X_train[:,i] = my_image[:,0]
        
    else:
        my_image = "dog."+str((i+1-n_cat))+".jpg" # change this to the name of your image file 
        fname = "training_set/dogs/" + my_image
        image = np.array(ndimage.imread(fname, flatten=False))
        my_image = scipy.misc.imresize(image, size=(num_px,num_px)).reshape((num_px*num_px*3,1))
        label_y[0][i] = 0
        X_train[:,i] = my_image[:,0]
    

#     fname = "test_set/cats/" + my_image
#     image = np.array(ndimage.imread(fname, flatten=False))
#     my_image = scipy.misc.imresize(image, size=(num_px,num_px)).reshape((num_px*num_px*3,1))
    # my_predicted_image = %precisiondict(my_image, my_label_y, parameters)
# print ("y = " + str(np.squeeze(my_predicted_image)) + ", your L-layer model predicts a \"" + 
#        classes[int(np.squeeze(my_predicted_image)),].decode("utf-8") +  "\" picture.")

# print(train_y[1:10])

200


In [68]:
print(m)
print(X_train.shape)
print(label_y.shape)

200
(12288, 200)
(1, 200)


In [79]:
train_x = X_train/255
train_y = label_y

In [80]:
# print(X_train)
# print(train_x)


In [112]:

def initialize_parameters_deep(layer_dims):
    """
    Arguments:
    layer_dims -- python array (list) containing the dimensions of each layer in our network
    
    Returns:
    parameters -- python dictionary containing your parameters "W1", "b1", ..., "WL", "bL":
                    Wl -- weight matrix of shape (layer_dims[l], layer_dims[l-1])
                    bl -- bias vector of shape (layer_dims[l], 1)
    """
    
    parameters = {}
    L = len(layer_dims)            # number of layers in the network

    for l in range(1, L):
        ### START CODE HERE ### (≈ 2 lines of code)
        parameters['W' + str(l)] = np.random.randn(layer_dims[l],layer_dims[l-1])*0.01
        parameters['b' + str(l)] = np.random.randn(layer_dims[l],1)
        ### END CODE HERE ###
        
        assert(parameters['W' + str(l)].shape == (layer_dims[l], layer_dims[l-1]))
        assert(parameters['b' + str(l)].shape == (layer_dims[l], 1))

        
    return parameters

In [113]:

def linear_forward(A, W, b):
    """
    Implement the linear part of a layer's forward propagation.

    Arguments:
    A -- activations from previous layer (or input data): (size of previous layer, number of examples)
    W -- weights matrix: numpy array of shape (size of current layer, size of previous layer)
    b -- bias vector, numpy array of shape (size of the current layer, 1)

    Returns:
    Z -- the input of the activation function, also called pre-activation parameter 
    cache -- a python dictionary containing "A", "W" and "b" ; stored for computing the backward pass efficiently
    """
    
    ### START CODE HERE ### (≈ 1 line of code)
    Z = np.dot(W,A)+b
    ### END CODE HERE ###
    
    assert(Z.shape == (W.shape[0], A.shape[1]))
    cache = (A, W, b)
    
    return Z, cache

In [114]:

def linear_activation_forward(A_prev, W, b, activation):
    """
    Implement the forward propagation for the LINEAR->ACTIVATION layer

    Arguments:
    A_prev -- activations from previous layer (or input data): (size of previous layer, number of examples)
    W -- weights matrix: numpy array of shape (size of current layer, size of previous layer)
    b -- bias vector, numpy array of shape (size of the current layer, 1)
    activation -- the activation to be used in this layer, stored as a text string: "sigmoid" or "relu"

    Returns:
    A -- the output of the activation function, also called the post-activation value 
    cache -- a python dictionary containing "linear_cache" and "activation_cache";
             stored for computing the backward pass efficiently
    """
    
    if activation == "sigmoid":
        # Inputs: "A_prev, W, b". Outputs: "A, activation_cache".
        ### START CODE HERE ### (≈ 2 lines of code)
        Z, linear_cache = linear_forward(A_prev,W,b)
        A, activation_cache = sigmoid(Z)
        ### END CODE HERE ###
    
    elif activation == "relu":
        # Inputs: "A_prev, W, b". Outputs: "A, activation_cache".
        ### START CODE HERE ### (≈ 2 lines of code)
        Z, linear_cache = linear_forward(A_prev,W,b)
        A, activation_cache = relu(Z)
        ### END CODE HERE ###
    
    assert (A.shape == (W.shape[0], A_prev.shape[1]))
    cache = (linear_cache, activation_cache)

    return A, cache

In [115]:

def L_model_forward(X, parameters):
    """
    Implement forward propagation for the [LINEAR->RELU]*(L-1)->LINEAR->SIGMOID computation
    
    Arguments:
    X -- data, numpy array of shape (input size, number of examples)
    parameters -- output of initialize_parameters_deep()
    
    Returns:
    AL -- last post-activation value
    caches -- list of caches containing:
                every cache of linear_relu_forward() (there are L-1 of them, indexed from 0 to L-2)
                the cache of linear_sigmoid_forward() (there is one, indexed L-1)
    """

    caches = []
    A = X
    L = len(parameters) // 2                  # number of layers in the neural network
    
    # Implement [LINEAR -> RELU]*(L-1). Add "cache" to the "caches" list.
    for l in range(1, L):
        A_prev = A 
        ### START CODE HERE ### (≈ 2 lines of code)
        A, cache = linear_activation_forward(A_prev,parameters['W'+str(l)],parameters['b' + str(l)],"relu")
        caches.append(cache)
        ### END CODE HERE ###
    
    # Implement LINEAR -> SIGMOID. Add "cache" to the "caches" list.
    ### START CODE HERE ### (≈ 2 lines of code)
    AL, cache = linear_activation_forward(A,parameters['W'+str(L)],parameters['b'+str(L)],"sigmoid")
    caches.append(cache)
    ### END CODE HERE ###
    
    assert(AL.shape == (1,X.shape[1]))
            
    return AL, caches

In [116]:

def compute_cost(AL, Y):
    """
    Implement the cost function defined by equation (7).

    Arguments:
    AL -- probability vector corresponding to your label predictions, shape (1, number of examples)
    Y -- true "label" vector (for example: containing 0 if non-cat, 1 if cat), shape (1, number of examples)

    Returns:
    cost -- cross-entropy cost
    """
    
    m = Y.shape[1]

    # Compute loss from aL and y.
    ### START CODE HERE ### (≈ 1 lines of code)
    logprobs = np.multiply(np.log(AL), Y) + np.multiply((1 - Y), np.log(1 - AL))
    cost = -1/m*np.sum(logprobs)
    ### END CODE HERE ###
    
    cost = np.squeeze(cost)      # To make sure your cost's shape is what we expect (e.g. this turns [[17]] into 17).
    assert(cost.shape == ())
    
    return cost

In [117]:

def linear_backward(dZ, cache):
    """
    Implement the linear portion of backward propagation for a single layer (layer l)

    Arguments:
    dZ -- Gradient of the cost with respect to the linear output (of current layer l)
    cache -- tuple of values (A_prev, W, b) coming from the forward propagation in the current layer

    Returns:
    dA_prev -- Gradient of the cost with respect to the activation (of the previous layer l-1), same shape as A_prev
    dW -- Gradient of the cost with respect to W (current layer l), same shape as W
    db -- Gradient of the cost with respect to b (current layer l), same shape as b
    """
    A_prev, W, b = cache
    m = A_prev.shape[1]

    ### START CODE HERE ### (≈ 3 lines of code)
    dW = 1/m*np.dot(dZ,A_prev.T)
    db =  1/m*(np.sum(dZ,axis=1, keepdims=True))
    dA_prev = np.dot(W.T,dZ)
    ### END CODE HERE ###
    
    assert (dA_prev.shape == A_prev.shape)
    assert (dW.shape == W.shape)
    assert (db.shape == b.shape)
    
    return dA_prev, dW, db

In [118]:

def linear_activation_backward(dA, cache, activation):
    """
    Implement the backward propagation for the LINEAR->ACTIVATION layer.
    
    Arguments:
    dA -- post-activation gradient for current layer l 
    cache -- tuple of values (linear_cache, activation_cache) we store for computing backward propagation efficiently
    activation -- the activation to be used in this layer, stored as a text string: "sigmoid" or "relu"
    
    Returns:
    dA_prev -- Gradient of the cost with respect to the activation (of the previous layer l-1), same shape as A_prev
    dW -- Gradient of the cost with respect to W (current layer l), same shape as W
    db -- Gradient of the cost with respect to b (current layer l), same shape as b
    """
    linear_cache, activation_cache = cache
    
    if activation == "relu":
        ### START CODE HERE ### (≈ 2 lines of code)
        dZ = relu_backward(dA,activation_cache)
        dA_prev, dW, db = linear_backward(dZ,linear_cache)
        ### END CODE HERE ###
        
    elif activation == "sigmoid":
        ### START CODE HERE ### (≈ 2 lines of code)
        dZ = sigmoid_backward(dA,activation_cache)
        dA_prev, dW, db = linear_backward(dZ,linear_cache)
        ### END CODE HERE ###
    
    return dA_prev, dW, db

In [125]:
def L_model_backward(AL, Y, caches):
    """
    Implement the backward propagation for the [LINEAR->RELU] * (L-1) -> LINEAR -> SIGMOID group
    
    Arguments:
    AL -- probability vector, output of the forward propagation (L_model_forward())
    Y -- true "label" vector (containing 0 if non-cat, 1 if cat)
    caches -- list of caches containing:
                every cache of linear_activation_forward() with "relu" (it's caches[l], for l in range(L-1) i.e l = 0...L-2)
                the cache of linear_activation_forward() with "sigmoid" (it's caches[L-1])
    
    Returns:
    grads -- A dictionary with the gradients
             grads["dA" + str(l)] = ... 
             grads["dW" + str(l)] = ...
             grads["db" + str(l)] = ... 
    """
    grads = {}
    L = len(caches) # the number of layers
    m = AL.shape[1]
    Y = Y.reshape(AL.shape) # after this line, Y is the same shape as AL
    
    # Initializing the backpropagation
    ### START CODE HERE ### (1 line of code)
    dAL = - (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL))
    ### END CODE HERE ###
    
    # Lth layer (SIGMOID -> LINEAR) gradients. Inputs: "AL, Y, caches". Outputs: "grads["dAL"], grads["dWL"], grads["dbL"]
    ### START CODE HERE ### (approx. 2 lines)
    current_cache = caches[-1]
    grads["dA" + str(L)], grads["dW" + str(L)], grads["db" + str(L)] = linear_activation_backward(dAL, current_cache, activation="sigmoid")
    ### END CODE HERE ###
    
    for l in reversed(range(L-1)):
        # lth layer: (RELU -> LINEAR) gradients.
        # Inputs: "grads["dA" + str(l + 2)], caches". Outputs: "grads["dA" + str(l + 1)] , grads["dW" + str(l + 1)] , grads["db" + str(l + 1)] 
        ### START CODE HERE ### (approx. 5 lines)
        current_cache = caches[l]
        
        dA_prev_temp, dW_temp, db_temp = linear_activation_backward(grads["dA" + str(l + 2)], current_cache, activation="relu")
        grads["dA" + str(l + 1)] = dA_prev_temp
        grads["dW" + str(l + 1)] = dW_temp
        grads["db" + str(l + 1)] = db_temp
        ### END CODE HERE ###

    return grads

In [126]:

def update_parameters(parameters, grads, learning_rate):
    """
    Update parameters using gradient descent
    
    Arguments:
    parameters -- python dictionary containing your parameters 
    grads -- python dictionary containing your gradients, output of L_model_backward
    
    Returns:
    parameters -- python dictionary containing your updated parameters 
                  parameters["W" + str(l)] = ... 
                  parameters["b" + str(l)] = ...
    """
    
    L = len(parameters) // 2 # number of layers in the neural network
    # Update rule for each parameter. Use a for loop.
    ### START CODE HERE ### (≈ 3 lines of code)
    for i in range(1,L+1):
        parameters["W" + str(i)] = parameters["W" + str(i)] - learning_rate*grads["dW"+str(i)]
        parameters["b" + str(i)] = parameters["b" + str(i)] - learning_rate*grads["db"+str(i)]
        
    ### END CODE HERE ###
    return parameters

In [127]:
n_x = 12288     # num_px * num_px * 3
layers_dims = [12288,20,7,5,1]

In [140]:

def L_layer_model(X, Y, layers_dims, learning_rate = 0.9, num_iterations = 3000, print_cost=False):#lr was 0.009
    """
    Implements a L-layer neural network: [LINEAR->RELU]*(L-1)->LINEAR->SIGMOID.
    
    Arguments:
    X -- data, numpy array of shape (number of examples, num_px * num_px * 3)
    Y -- true "label" vector (containing 0 if cat, 1 if non-cat), of shape (1, number of examples)
    layers_dims -- list containing the input size and each layer size, of length (number of layers + 1).
    learning_rate -- learning rate of the gradient descent update rule
    num_iterations -- number of iterations of the optimization loop
    print_cost -- if True, it prints the cost every 100 steps
    
    Returns:
    parameters -- parameters learnt by the model. They can then be used to predict.
    """

    
    costs = []                         # keep track of cost
    
    # Parameters initialization.
    ### START CODE HERE ###
    parameters = initialize_parameters_deep(layers_dims)
#     print(parameters)
    ### END CODE HERE ###
    
    # Loop (gradient descent)
    for i in range(0, num_iterations):

        # Forward propagation: [LINEAR -> RELU]*(L-1) -> LINEAR -> SIGMOID.
        ### START CODE HERE ### (≈ 1 line of code)
        AL, caches = L_model_forward(X,parameters)
        ### END CODE HERE ###
        
        # Compute cost.
        ### START CODE HERE ### (≈ 1 line of code)
        cost = compute_cost(AL,Y)
        ### END CODE HERE ###
    
        # Backward propagation.
        ### START CODE HERE ### (≈ 1 line of code)
        grads = L_model_backward(AL,Y,caches)
        ### END CODE HERE ###
 
        # Update parameters.
        ### START CODE HERE ### (≈ 1 line of code)
        parameters = update_parameters(parameters,grads,learning_rate)
        ### END CODE HERE ###
                
        # Print the cost every 100 training example
        if print_cost and i % 100 == 0:
            print ("Cost after iteration %i: %f" %(i, cost))
        if print_cost and i % 100 == 0:
            costs.append(cost)
            
    # plot the cost
    plt.plot(np.squeeze(costs))
    plt.ylabel('cost')
    plt.xlabel('iterations (per tens)')
    plt.title("Learning rate =" + str(learning_rate))
    plt.show()
    
    return parameters

In [141]:
train_x.shape

(12288, 200)

In [142]:
parameters = L_layer_model(train_x,train_y , layers_dims, num_iterations = 2500, print_cost = True)

[[0.16226999 0.16226999 0.16226994 0.16226998 0.16227016 0.16227
  0.16227005 0.1622701  0.16227009 0.16227002 0.16227006 0.1622701
  0.16227004 0.16227    0.16227003 0.16227007 0.1622701  0.16227
  0.16226991 0.16227006 0.16226997 0.16227006 0.16226995 0.16227004
  0.16226999 0.16227005 0.16227003 0.16227008 0.16227008 0.16227012
  0.16227006 0.1622701  0.16227007 0.16227004 0.16226997 0.16227012
  0.16227006 0.16226999 0.16227005 0.16227001 0.16226994 0.16227011
  0.16226995 0.16227008 0.16227    0.16227003 0.1622701  0.16227006
  0.16227006 0.16227004 0.16226998 0.16227004 0.16226995 0.16227002
  0.16227004 0.16227002 0.16227005 0.1622701  0.16227005 0.16227004
  0.16226996 0.16227005 0.16227001 0.16227002 0.16227007 0.16227006
  0.16227008 0.16227    0.16227006 0.16227002 0.16226986 0.16226997
  0.16226988 0.16227011 0.16227003 0.16227001 0.16227017 0.16227002
  0.16227001 0.16227007 0.16227    0.16226998 0.16227006 0.16227006
  0.16227005 0.16226995 0.16226999 0.16227005 0.1622699

[[0.2721515  0.27215075 0.27214568 0.27214993 0.27217397 0.27215235
  0.27215919 0.2721664  0.27216413 0.27215455 0.27216055 0.27216516
  0.27215826 0.27215195 0.27215595 0.27216176 0.27216585 0.2721524
  0.27213991 0.27216115 0.27214852 0.27215993 0.27214625 0.27215827
  0.27215078 0.27215811 0.27215663 0.27216302 0.2721626  0.27216919
  0.27216034 0.27216675 0.27216149 0.27215825 0.27214885 0.27216834
  0.27216042 0.27215084 0.2721583  0.27215414 0.27214474 0.27216701
  0.27214558 0.27216247 0.27215241 0.27215656 0.27216678 0.27215991
  0.27216025 0.27215752 0.27214958 0.2721571  0.27214613 0.27215516
  0.27215818 0.27215489 0.27215836 0.27216559 0.27215885 0.27215783
  0.27214741 0.27215921 0.2721529  0.27215496 0.27216184 0.2721598
  0.27216307 0.27215323 0.27216094 0.27215451 0.27213343 0.27214856
  0.27213635 0.27216678 0.27215628 0.27215428 0.27217559 0.27215488
  0.27215416 0.27216173 0.27215182 0.27215018 0.27216037 0.27215979
  0.27215895 0.27214492 0.2721502  0.27215899 0.27

[[0.41700256 0.41700173 0.41699146 0.41700372 0.41713814 0.41701903
  0.41703774 0.41709437 0.41707262 0.41701977 0.41705092 0.41707422
  0.41704181 0.41699281 0.41702319 0.41706591 0.41706863 0.41700177
  0.41694345 0.41706339 0.41698773 0.41703604 0.41699309 0.41704141
  0.41698779 0.41702197 0.41702864 0.41707047 0.41706154 0.41710896
  0.4170579  0.4170975  0.41705567 0.41704301 0.41699475 0.41710604
  0.41704793 0.41700368 0.41702605 0.41702838 0.41696625 0.41709122
  0.41697857 0.41705778 0.41701037 0.41703434 0.41710766 0.4170428
  0.41705298 0.41703155 0.41698914 0.41702053 0.41697048 0.41701867
  0.41704004 0.41702365 0.41703484 0.4170895  0.41705439 0.41702883
  0.41697871 0.41704389 0.417006   0.41702107 0.41705184 0.41704848
  0.41707635 0.41703112 0.41705284 0.41702204 0.41691578 0.41698293
  0.41694049 0.41708277 0.41702841 0.41702637 0.4171275  0.41701599
  0.41702661 0.41705986 0.41699709 0.41699706 0.41705223 0.41704542
  0.41705544 0.41696401 0.41698824 0.41704738 0.4

[[0.47001872 0.47002143 0.47001995 0.47002874 0.47025165 0.47005551
  0.47007098 0.47017619 0.47013533 0.47004764 0.47009666 0.47013288
  0.4700869  0.4699943  0.47004934 0.47012704 0.47011725 0.47001394
  0.46992759 0.47012609 0.46999819 0.47006282 0.47002235 0.47008359
  0.46998779 0.47003791 0.47005987 0.47013372 0.47011596 0.47019977
  0.47011578 0.47018491 0.47010253 0.470088   0.47001323 0.47019693
  0.47008837 0.47002731 0.47004612 0.47007068 0.46996311 0.47016679
  0.46998762 0.47010497 0.47003473 0.47007492 0.47020696 0.47007982
  0.47010388 0.47006614 0.46999585 0.4700383  0.4699656  0.47004238
  0.47007978 0.47005752 0.47006818 0.47016819 0.47011487 0.47005644
  0.46997976 0.47008505 0.47002229 0.4700493  0.47009504 0.47009588
  0.47014924 0.47008325 0.47010097 0.47005317 0.46988856 0.46998465
  0.46993599 0.47014556 0.4700638  0.47006491 0.47021619 0.47003733
  0.47006441 0.47011338 0.47000414 0.47001362 0.47010004 0.47008616
  0.47011511 0.46995409 0.46998972 0.47009559 0.

[[0.48517183 0.48517644 0.48517989 0.48518569 0.48544416 0.4852167
  0.48523074 0.48535482 0.48530708 0.48520521 0.48526093 0.48530212
  0.48525254 0.48514201 0.48520617 0.48529693 0.48528287 0.48516576
  0.48506935 0.48529849 0.48515074 0.4852192  0.48518401 0.48524664
  0.48513525 0.48519057 0.48521838 0.48530541 0.48528532 0.48538147
  0.4852856  0.4853665  0.4852664  0.48525259 0.4851688  0.48537903
  0.4852496  0.48518552 0.48520039 0.48523482 0.48511059 0.48534259
  0.48513833 0.48526998 0.48519173 0.48523885 0.48539321 0.48524074
  0.4852704  0.4852285  0.48514606 0.48519165 0.48511136 0.48519838
  0.48524144 0.48521892 0.48522842 0.48534536 0.48528722 0.48521454
  0.4851277  0.48524741 0.48517596 0.48520713 0.48525962 0.48526204
  0.48532553 0.48525197 0.48526724 0.48521181 0.48502644 0.48513238
  0.48508321 0.48531539 0.4852262  0.48522718 0.48539634 0.48519244
  0.48522578 0.48528067 0.48515349 0.48516856 0.48526496 0.48524791
  0.4852854  0.485097   0.48513694 0.4852616  0.4

...

[[0.49963477 0.49963477 0.49963477 0.49963477 0.49963477 0.49963477
  0.49963477 0.49963477 0.49963477 0.49963477 0.49963477 0.49963477
  0.49963477 0.49963477 0.49963477 0.49963477 0.49963477 0.49963477
  0.49963477 0.49963477 0.49963477 0.49963477 0.49963477 0.49963477
  0.49963477 0.49963477 0.49963477 0.49963477 0.49963477 0.49963477
  0.49963477 0.49963477 0.49963477 0.49963477 0.49963477 0.49963477
  0.49963477 0.49963477 0.49963477 0.49963477 0.49963477 0.49963477
  0.49963477 0.49963477 0.49963477 0.49963477 0.49963477 0.49963477
  0.49963477 0.49963477 0.49963477 0.49963477 0.49963477 0.49963477
  0.49963477 0.49963477 0.49963477 0.49963477 0.49963477 0.49963477
  0.49963477 0.49963477 0.49963477 0.49963477 0.49963477 0.49963477
  0.49963477 0.49963477 0.49963477 0.49963477 0.49963477 0.49963477
  0.49963477 0.49963477 0.49963477 0.49963477 0.49963477 0.49963477
  0.49963477 0.49963477 0.49963477 0.49963477 0.49963477 0.49963477
  0.49963477 0.49963477 0.49963477 0.49963477 0.

[[0.49980979 0.49980979 0.49980979 0.49980979 0.49980979 0.49980979
  0.49980979 0.49980979 0.49980979 0.49980979 0.49980979 0.49980979
  0.49980979 0.49980979 0.49980979 0.49980979 0.49980979 0.49980979
  0.49980979 0.49980979 0.49980979 0.49980979 0.49980979 0.49980979
  0.49980979 0.49980979 0.49980979 0.49980979 0.49980979 0.49980979
  0.49980979 0.49980979 0.49980979 0.49980979 0.49980979 0.49980979
  0.49980979 0.49980979 0.49980979 0.49980979 0.49980979 0.49980979
  0.49980979 0.49980979 0.49980979 0.49980979 0.49980979 0.49980979
  0.49980979 0.49980979 0.49980979 0.49980979 0.49980979 0.49980979
  0.49980979 0.49980979 0.49980979 0.49980979 0.49980979 0.49980979
  0.49980979 0.49980979 0.49980979 0.49980979 0.49980979 0.49980979
  0.49980979 0.49980979 0.49980979 0.49980979 0.49980979 0.49980979
  0.49980979 0.49980979 0.49980979 0.49980979 0.49980979 0.49980979
  0.49980979 0.49980979 0.49980979 0.49980979 0.49980979 0.49980979
  0.49980979 0.49980979 0.49980979 0.49980979 0.

[[0.49988714 0.49988714 0.49988714 0.49988714 0.49988714 0.49988714
  0.49988714 0.49988714 0.49988714 0.49988714 0.49988714 0.49988714
  0.49988714 0.49988714 0.49988714 0.49988714 0.49988714 0.49988714
  0.49988714 0.49988714 0.49988714 0.49988714 0.49988714 0.49988714
  0.49988714 0.49988714 0.49988714 0.49988714 0.49988714 0.49988714
  0.49988714 0.49988714 0.49988714 0.49988714 0.49988714 0.49988714
  0.49988714 0.49988714 0.49988714 0.49988714 0.49988714 0.49988714
  0.49988714 0.49988714 0.49988714 0.49988714 0.49988714 0.49988714
  0.49988714 0.49988714 0.49988714 0.49988714 0.49988714 0.49988714
  0.49988714 0.49988714 0.49988714 0.49988714 0.49988714 0.49988714
  0.49988714 0.49988714 0.49988714 0.49988714 0.49988714 0.49988714
  0.49988714 0.49988714 0.49988714 0.49988714 0.49988714 0.49988714
  0.49988714 0.49988714 0.49988714 0.49988714 0.49988714 0.49988714
  0.49988714 0.49988714 0.49988714 0.49988714 0.49988714 0.49988714
  0.49988714 0.49988714 0.49988714 0.49988714 0.

[[0.49994123 0.49994123 0.49994123 0.49994123 0.49994123 0.49994123
  0.49994123 0.49994123 0.49994123 0.49994123 0.49994123 0.49994123
  0.49994123 0.49994123 0.49994123 0.49994123 0.49994123 0.49994123
  0.49994123 0.49994123 0.49994123 0.49994123 0.49994123 0.49994123
  0.49994123 0.49994123 0.49994123 0.49994123 0.49994123 0.49994123
  0.49994123 0.49994123 0.49994123 0.49994123 0.49994123 0.49994123
  0.49994123 0.49994123 0.49994123 0.49994123 0.49994123 0.49994123
  0.49994123 0.49994123 0.49994123 0.49994123 0.49994123 0.49994123
  0.49994123 0.49994123 0.49994123 0.49994123 0.49994123 0.49994123
  0.49994123 0.49994123 0.49994123 0.49994123 0.49994123 0.49994123
  0.49994123 0.49994123 0.49994123 0.49994123 0.49994123 0.49994123
  0.49994123 0.49994123 0.49994123 0.49994123 0.49994123 0.49994123
  0.49994123 0.49994123 0.49994123 0.49994123 0.49994123 0.49994123
  0.49994123 0.49994123 0.49994123 0.49994123 0.49994123 0.49994123
  0.49994123 0.49994123 0.49994123 0.49994123 0.

KeyboardInterrupt: 

In [32]:
pred_train = predict(train_x, train_y, parameters)

Accuracy: 0.6701250000000002


In [34]:
path, dirs, cat_files_test = next(os.walk("test_set/cats/"))
file_count_cat = len(cat_files_test)
print(file_count_cat)

path, dirs, dog_files_test = next(os.walk("test_set/dogs/"))
file_count_dog = len(dog_files_test)
print(file_count_dog)
    

1001
1001


In [55]:
n_cat_test = file_count_cat-1
n_dog_test = file_count_dog-1

#Variables
m_test = n_cat_test + n_dog_test               # Number of train ex.
print(m_test)
num_px = 64
y_test = np.zeros((1,m_test))
X_test = np.zeros((12288,m_test))
# Input Dataset from the folder

for i in range(4000,m_test+4000):
#     print(i%n_cat)
    if i<n_cat_test+4000:
        my_image = "cat."+str(i+1)+".jpg" # change this to the name of your image file 
        fname = "test_set/cats/" + my_image
        image = np.array(ndimage.imread(fname, flatten=False))
        my_image = scipy.misc.imresize(image, size=(num_px,num_px)).reshape((num_px*num_px*3,1))
        y_test[0][i%4000] = 1 # the true class of your image (1 -> cat, 0 -> non-cat)
#         print(my_image)
        X_test[:,i%4000] = my_image[:,0]
#         print(X_train)
        
    else:
        my_image = "dog."+str(i+1-n_cat_test)+".jpg" # change this to the name of your image file 
        fname = "test_set/dogs/" + my_image
        image = np.array(ndimage.imread(fname, flatten=False))
        my_image = scipy.misc.imresize(image, size=(num_px,num_px)).reshape((num_px*num_px*3,1))
        y_test[0][i%4000] = 0
        X_test[:,i%4000] = my_image[:,0]
    

#     fname = "test_set/cats/" + my_image
#     image = np.array(ndimage.imread(fname, flatten=False))
#     my_image = scipy.misc.imresize(image, size=(num_px,num_px)).reshape((num_px*num_px*3,1))
    # my_predicted_image = %precisiondict(my_image, my_label_y, parameters)
# print ("y = " + str(np.squeeze(my_predicted_image)) + ", your L-layer model predicts a \"" + 
#        classes[int(np.squeeze(my_predicted_image)),].decode("utf-8") +  "\" picture.")

# print(train_y[1:10])

2000


In [59]:
test_x = X_test
test_y = y_test 
pred_test = predict(test_x, test_y, parameters)


Accuracy: 0.5260000000000002
